In [ ]:
import yaml
import os
with open('configs/config.yaml','r') as f:
    cfg = yaml.safe_load(f)

datasets_cfg = cfg['dataset']
models_cfg = cfg['model']
train_cfg = cfg['train']
optim_cfg = cfg['optim']
checkpoint_cfg = cfg['checkpoint']
output_cfg = cfg['output']
trained_models = os.listdir('checkpoints')
if f'{models_cfg['name']}.pth' not in trained_models:
    print('s')

In [15]:
import yaml
import os
with open('configs/config.yaml','r') as f:
    cfg = yaml.safe_load(f)

In [ ]:
import pandas as pd
a= 
pd.DataFrame(a)

,epoch,train_loss,train_acc,val_loss,val_acc
0,1,6,3,4,5


In [ ]:
import torch
from datasets import get_dataset
from utils.get_transforms import get_transforms
from utils.dataloader import get_dataloaders
from models.model import get_model
from utils.train_utils import train_loop
from utils.get_optim import get_optim
from tqdm import tqdm
from torchvision import models
from torch.nn import CrossEntropyLoss
import torch
import yaml
from torchvision import models
from torch.nn import CrossEntropyLoss
from tqdm import tqdm

# ... توابع شما (get_dataset, get_dataloaders, و غیره) ...

# --- تنظیمات اولیه ---
with open('configs/config.yaml', 'r') as f:
    cfg = yaml.safe_load(f)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# --- ۱. بارگذاری مدل و TRANSFORM صحیح ---
weights = models.Swin_T_Weights.IMAGENET1K_V1
model = models.swin_t(weights=weights)
auto_transforms = weights.transforms() # استفاده از transform استاندارد

# --- آماده‌سازی دیتاست ---
dataset_name = 'cub' # مثال
ds_cfg = cfg['datasets'][dataset_name]
num_classes = 200 # مثلاً 200

train_dataset = get_dataset(dataset_name, ds_cfg, split='train', transform=auto_transforms)
test_dataset = get_dataset(dataset_name, ds_cfg, split='test', transform=auto_transforms)
dataset_dict = {'train': train_dataset, 'test': test_dataset}
dataloaders = get_dataloaders(dataset_dict, ds_cfg, shuffle_train=True)

print(f"Train dataset size: {len(dataset_dict['train'])}, Test dataset size: {len(dataset_dict['test'])}")

# --- ۲. استراتژی FINE-TUNING مرحله اول: فقط آموزش HEAD ---

# فریز کردن تمام لایه‌های backbone
for param in model.parameters():
    param.requires_grad = False

# جایگزینی head و اطمینان از اینکه قابل آموزش است
in_features = model.head.in_features
model.head = torch.nn.Linear(in_features, num_classes)
# پارامترهای model.head به طور خودکار requires_grad=True هستند

model.to(device)

# تعریف optimizer فقط برای پارامترهای head
optimizer = torch.optim.AdamW(model.head.parameters(), lr=1e-3, weight_decay=1e-2)
criterion = CrossEntropyLoss()

# چاپ اطلاعات برای اطمینان
num_params_trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Model loaded for Stage 1: Training the head only.")
print("Head:", model.head)
print(f"Trainable parameters: {num_params_trainable}")

# --- حلقه آموزش ساده برای تست ---
# شما باید این را در تابع train_loop خود ادغام کنید
model.train()
for epoch in range(5): # برای ۵ ایپاک فقط head را آموزش دهید
    for xb, yb, paths in tqdm(dataloaders['train'], desc=f"Epoch {epoch+1}"):
        xb, yb = xb.to(device), yb.to(device)
        
        optimizer.zero_grad()
        pred = model(xb)
        loss = criterion(pred, yb)
        loss.backward()
        optimizer.step()

    # ارزیابی در انتهای هر ایپاک (توصیه می‌شود)
    model.eval()
    total_correct = 0
    total_samples = 0
    with torch.no_grad():
        for xb, yb, paths in dataloaders['test']:
            xb, yb = xb.to(device), yb.to(device)
            pred = model(xb)
            preds = torch.argmax(pred, 1)
            total_correct += (preds == yb).sum().item()
            total_samples += yb.size(0)
    accuracy = total_correct / total_samples 
    print(f"Epoch {epoch+1} Test Accuracy: {accuracy:.4f}")
    model.train()

# پس از این مرحله، دقت مدل باید به طور قابل توجهی افزایش یافته باشد.
# سپس می‌توانید مرحله دوم (unfreeze و آموزش با lr پایین) را اجرا کنید.


Train dataset size: 5994, Test dataset size: 5794
Model loaded for Stage 1: Training the head only.
Head: Linear(in_features=768, out_features=200, bias=True)
Trainable parameters: 153800


Epoch 1: 100%|██████████| 188/188 [00:27<00:00,  6.73it/s]


Epoch 1 Test Accuracy: 0.5186


Epoch 2: 100%|██████████| 188/188 [00:27<00:00,  6.78it/s]


Epoch 2 Test Accuracy: 0.6391


Epoch 3:   0%|          | 0/188 [00:07<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
import torchvision

from torchvision.models import swin_t, Swin_T_Weights
model = swin_t(weights=Swin_T_Weights.IMAGENET1K_V1)
in_features = model.head.in_features
model.head = torch.nn.Linear(in_features, 200)



Model loaded. Head: Linear(in_features=768, out_features=200, bias=True)
Trainable parameters: 27673154


In [2]:
import glob
import os
import pandas as pd
from collections import Counter
import numpy as np
num_classes = 200

csvs =[]
vote_dict = {}
conf_dict = {}
label_dict = {}
percentage_dict = {}
hard_results = []
soft_results = []

df1 = pd.read_parquet('outputs/swin_t_cub/preds_val_cub_dataset.parquet',engine='pyarrow')
df2 = pd.read_parquet('outputs/efficientnet_v2_s_cub/preds_val_cub_dataset.parquet',engine='pyarrow')
c =61
df3 = pd.read_csv('outputs/ensemble_cub_soft.csv') 
len(df3[df3['ensemble_pred']==df3['label']])/len(df3)

FileNotFoundError: [Errno 2] No such file or directory: 'outputs/ensemble_cub_soft.csv'

In [ ]:
csvs =[]
vote_dict = {}
conf_dict = {}
label_dict = {}
percentage_dict = {}
hard_results = []
soft_results = []
folders=os.listdir('outputs')
dataset_name = 'cub'
for folder in folders:
    if os.path.isdir(f"outputs/{folder}") :
        csvs.append(glob.glob(f"outputs/{folder}/*_{dataset_name}_dataset.parquet").pop())
for csv in csvs:
    df = pd.read_parquet(csv,engine='pyarrow')
    for i,row in df.iterrows():
        path = row['image_path']
        pred = row['prediction']
        conf = row['confidence']
        label = row['label']
        percentage = row['percentage']
        vote_dict.setdefault(path,[]).append(pred)
        conf_dict.setdefault(path,[]).append(conf)
        percentage_dict.setdefault(path,[]).append(percentage)
        label_dict[path] = label

majority = 0
for path in vote_dict:
    preds = vote_dict[path]
    confs = conf_dict[path]
    percentages = percentage_dict[path]
    hard = Counter(preds).most_common(1)[0][0]
    
    if hard == label_dict[path]:
        majority+=1
    probs = np.zeros(num_classes)
    for pred,percent in zip(preds,percentages):
        probs += percent
    soft = np.argmax(probs)
    hard_results.append({'image_path':path,'label':label_dict[path],'ensemble_pred':hard,'voting':'hard'})
    soft_results.append({'img_path': path, 'label': label_dict[path], 'ensemble_pred': soft, 'voting': 'soft'})
soft_df = pd.DataFrame(soft_results)
hard_df = pd.DataFrame(hard_results)
print((majority/len(vote_dict) * 100))
print(len(soft_df[soft_df['label']==soft_df['ensemble_pred']])/len(soft_df) * 100)
print(len(hard_df[hard_df['label']==hard_df['ensemble_pred']])/len(hard_df) * 100)


35.20883672764929
36.79668622713152


KeyError: 'ensemble_pred'